In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
train = pd.read_csv("Train.csv")
target = pd.read_csv("Target.csv")

In [12]:
import pycountry

def get_iso3_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        return country.alpha_3
    except AttributeError:
        return None
    
train['ISO-3'] = train['Country'].apply(get_iso3_code)

fig = px.choropleth(train, 
                    locations='Country',
                    locationmode='ISO-3',
                    title='Carte de l\'Europe',
                    color_continuous_scale='Viridis',
                    color='food')

fig.show()

LookupError: 

In [3]:
train.head()

,Unnamed: 0,code,year,period,id,Country,tourists,venue,rate,food,glass,metal,other,paper,plastic,leather,green_waste,waste_recycling
0,6,CAS_7723-14-0,2009,2009-01--2009-12,ES014MSPFES-014-NR-038-000-02-02,Spain,50941692.0,4.0,87.158924,49.0,8.0,3.0,14.0,15.0,9.0,0.0,0.0,16.84
1,10,CAS_7723-14-0,2009,2009-01--2009-12,FRGR0220,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2,17,CAS_14265-44-2,2009,2009-01--2009-12,FRGR1496,France,71176346.0,4.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
3,24,CAS_7723-14-0,2009,2009-01--2009-12,FRDR174,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
4,34,CAS_7723-14-0,2009,2009-01--2009-12,FRGR1022,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26


In [9]:
import plotly.express as px

In [10]:
fig = px.choropleth(train, 
                    locations=train['Country'], 
                    locationmode=train['Country'],
                    title='Carte de l\'Europe',
                    color_continuous_scale='Viridis',
                    color=train['food'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().